In [1]:
import sys
import pymol
_stdouterr = sys.stdout, sys.stderr
pymol.finish_launching(['/usr/bin/pymol', '-q'])
sys.stdout, sys.stderr = _stdouterr

# load something into the PyMOL window
from pymol import cmd


Gtk-Message: 12:19:32.621: Failed to load module "pk-gtk-module"
Cannot open file '/home/vit/Projects/cryptoshow-analysis/src/prediction-analysis/data/pymol/icons/icon2.svg', because: No such file or directory
Cannot open file '/home/vit/Projects/cryptoshow-analysis/src/prediction-analysis/data/pymol/icons/icon2.svg', because: No such file or directory


Could not read PyMOL stylesheet.
DEBUG: PYMOL_DATA='./data'
 Detected OpenGL version 4.6. Shaders available.
 Geometry shaders not available
 Detected GLSL version 4.60.
PyMOL>set fetch_path, /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files
 Setting: fetch_path set to /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files.


# visualizations without smoothing
visualize pure predictions in blue and smoothing in green

In [ ]:
import json
import sys, os
import numpy as np

from biotite.structure import get_residues
from biotite.structure.io.pdbx import get_structure
import biotite.structure.io.pdbx as pdbx
import biotite.database.rcsb as rcsb
import biotite
import pickle

DATASET = 'cryptobench'
CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'
PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/D-visualize/predictions/finetuning-without-smoothing'
SMOOTHED_PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/D-visualize/predictions/finetuning-with-smoothing'
DECISION_THRESHOLD = 0.7

with open(f'/home/vit/Projects/cryptoshow-analysis/datasets/cryptobench-dataset/folds/test.json', 'r') as json_file:
    dataset = json.load(json_file)

skip = True
for apo_structure, holo_structures in dataset.items():
    # if apo_structure != '5yqp' and apo_structure != '7nlx':
    #     continue
    # finished analysis at: '5wbmB' structure
    if skip:
        if apo_structure == '7e5q':
            skip = False
        else:
            continue
    # print(f'Processing {apo_structure} ...')
    binding_residues = set()
    chain_id = holo_structures[0]['apo_chain']

    # skip multichain structures
    if '-' in chain_id:
        continue

    for holo_structure in holo_structures:

        apo_pocket = holo_structure['apo_pocket_selection']
        
        new_apo_residues = [residue.split(
            '_')[1] for residue in apo_pocket]

        binding_residues.update(new_apo_residues)

    cif_file_path = rcsb.fetch(apo_structure, "cif", target_path=CIF_FILES)

    cif_file = pdbx.CIFFile.read(cif_file_path)

    auth = get_structure(cif_file, model=1)
    auth = auth[
            (auth.chain_id == chain_id) &
            (biotite.structure.filter_peptide_backbone(auth))]
    
    protein_id = f'{apo_structure}{chain_id}'
    # skip if no residues left
    if len(auth) == 0: 
        print(f'No residues left for {protein_id}')
        continue

    # filter to get correct chain; filter only for peptides
    auth_residues_only = get_residues(auth)

    predictions = np.load(f'{PREDICTIONS_PATH}/{protein_id}.npy') > DECISION_THRESHOLD
    
    with open(f'{SMOOTHED_PREDICTIONS_PATH}/{protein_id}.pkl', 'rb') as f:
        smoothed_predictions = pickle.load(f) 
    if len(smoothed_predictions) > 0:
        smoothed_predictions = np.concatenate(smoothed_predictions) 
        smoothed_predictions_array = np.zeros(len(auth_residues_only[0]), dtype=bool)
        smoothed_predictions_array[smoothed_predictions] = True
    else:
        smoothed_predictions_array = np.zeros(len(auth_residues_only[0]), dtype=bool)

    assert len(predictions) == len(auth_residues_only[0]), f"Length of auth residues and predictions do not match for {protein_id}: {len(auth_residues_only[0])} vs {len(predictions)}"
    predicted_binding_residues = auth_residues_only[0][predictions]
    predicted_smoothed_binding_residues = auth_residues_only[0][smoothed_predictions_array]
    true_positives = binding_residues.intersection([str(i) for i in predicted_binding_residues])
    
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)
    # Show smoothing predictions in green
    if len(predicted_smoothed_binding_residues) > 0:
        cmd.color('green',f'{protein_id} and resi {"+".join([str(i) for i in predicted_smoothed_binding_residues])}')

    # Show regular predictions in blue
    if len(predicted_binding_residues) > 0:
        cmd.color('blue',f'{protein_id} and resi {"+".join([str(i) for i in predicted_binding_residues])}')
    # cmd.color('blue', f'{protein_id} and resi {"+".join([str(i) for i in binding_residues])}')
    # if len(true_positives) > 0:
    #     cmd.color('green', f'{protein_id} and resi {"+".join([str(i) for i in true_positives])}')
    cmd.show('surface', protein_id)
    decision = input(">Press Enter for the next protein...\n")
    if decision.lower() == 'q':
        break
# 3n4u
# 7o1i

 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF


# visualizations with smoothing
Visualize everything colored uniformly

In [ ]:
import json
import pickle
import numpy as np

from biotite.structure import get_residues
from biotite.structure.io.pdbx import get_structure
import biotite.structure.io.pdbx as pdbx
import biotite.database.rcsb as rcsb
import biotite


DATASET = 'cryptobench'
CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'
PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/D-visualize/predictions/finetuning-with-smoothing'

DECISION_THRESHOLD = 0.7

with open(f'/home/vit/Projects/cryptoshow-analysis/datasets/cryptobench-dataset/folds/test.json', 'r') as json_file:
    dataset = json.load(json_file)

skip = False
for apo_structure, holo_structures in dataset.items():
    if apo_structure != '7e5q':
        continue
    # finished analysis at: '5wbmB' structure
    if skip:
        if apo_structure == '5uxa':
            skip = False
        else:
            continue
    # print(f'Processing {apo_structure} ...')
    binding_residues = set()
    chain_id = holo_structures[0]['apo_chain']

    # skip multichain structures
    if '-' in chain_id:
        continue

    for holo_structure in holo_structures:

        apo_pocket = holo_structure['apo_pocket_selection']
        
        new_apo_residues = [residue.split(
            '_')[1] for residue in apo_pocket]

        binding_residues.update(new_apo_residues)

    cif_file_path = rcsb.fetch(apo_structure, "cif", target_path=CIF_FILES)

    cif_file = pdbx.CIFFile.read(cif_file_path)

    auth = get_structure(cif_file, model=1)
    auth = auth[
            (auth.chain_id == chain_id) &
            (biotite.structure.filter_peptide_backbone(auth))]
    
    protein_id = f'{apo_structure}{chain_id}'
    # skip if no residues left
    if len(auth) == 0: 
        print(f'No residues left for {protein_id}')
        continue

    # filter to get correct chain; filter only for peptides
    auth_residues_only = get_residues(auth)

    with open(f'{PREDICTIONS_PATH}/{protein_id}.pkl', 'rb') as f:
        predictions = pickle.load(f) 
    predictions = np.concatenate(predictions) 
    predictions_array = np.zeros(len(auth_residues_only[0]), dtype=bool)
    predictions_array[predictions] = True

    assert len(predictions_array) == len(auth_residues_only[0]), f"Length of auth residues and predictions do not match for {protein_id}: {len(auth_residues_only[0])} vs {len(predictions)}"
    predicted_binding_residues = auth_residues_only[0][predictions_array]
    true_positives = binding_residues.intersection([str(i) for i in predicted_binding_residues])
    
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)
    cmd.color('green',f'{protein_id} and resi {"+".join([str(i) for i in predicted_binding_residues])}')
    # cmd.color('blue', f'{protein_id} and resi {"+".join([str(i) for i in binding_residues])}')
    # if len(true_positives) > 0:
    #     cmd.color('green', f'{protein_id} and resi {"+".join([str(i) for i in true_positives])}')
    cmd.show('surface', protein_id)
    decision = input(">Press Enter for the next protein...\n")
    if decision.lower() == 'q':
        break

 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF


In [7]:
true_positives

set()

# For scheme
show cartoon with pocket in surface

In [4]:
import json
import sys, os
import numpy as np

from biotite.structure import get_residues
from biotite.structure.io.pdbx import get_structure
import biotite.structure.io.pdbx as pdbx
import biotite.database.rcsb as rcsb
import biotite
import pickle

DATASET = 'cryptobench'
CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'
PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/D-visualize/predictions/finetuning-without-smoothing'
SMOOTHED_PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/D-visualize/predictions/finetuning-with-smoothing'
DECISION_THRESHOLD = 0.7

with open(f'/home/vit/Projects/cryptoshow-analysis/datasets/cryptobench-dataset/folds/test.json', 'r') as json_file:
    dataset = json.load(json_file)

skip = True
for apo_structure, holo_structures in dataset.items():
    # if apo_structure != '5yqp' and apo_structure != '7nlx':
    #     continue
    # finished analysis at: '5wbmB' structure
    if skip:
        if apo_structure == '7e5q':
            skip = False
        else:
            continue
    # print(f'Processing {apo_structure} ...')
    binding_residues = set()
    chain_id = holo_structures[0]['apo_chain']

    # skip multichain structures
    if '-' in chain_id:
        continue

    for holo_structure in holo_structures:

        apo_pocket = holo_structure['apo_pocket_selection']
        
        new_apo_residues = [residue.split(
            '_')[1] for residue in apo_pocket]

        binding_residues.update(new_apo_residues)

    cif_file_path = rcsb.fetch(apo_structure, "cif", target_path=CIF_FILES)

    cif_file = pdbx.CIFFile.read(cif_file_path)

    auth = get_structure(cif_file, model=1)
    auth = auth[
            (auth.chain_id == chain_id) &
            (biotite.structure.filter_peptide_backbone(auth))]
    
    protein_id = f'{apo_structure}{chain_id}'
    # skip if no residues left
    if len(auth) == 0: 
        print(f'No residues left for {protein_id}')
        continue

    # filter to get correct chain; filter only for peptides
    auth_residues_only = get_residues(auth)

    predictions = np.load(f'{PREDICTIONS_PATH}/{protein_id}.npy') > DECISION_THRESHOLD
    
    with open(f'{SMOOTHED_PREDICTIONS_PATH}/{protein_id}.pkl', 'rb') as f:
        smoothed_predictions = pickle.load(f) 
    if len(smoothed_predictions) > 0:
        smoothed_predictions = np.concatenate(smoothed_predictions) 
        smoothed_predictions_array = np.zeros(len(auth_residues_only[0]), dtype=bool)
        smoothed_predictions_array[smoothed_predictions] = True
    else:
        smoothed_predictions_array = np.zeros(len(auth_residues_only[0]), dtype=bool)

    assert len(predictions) == len(auth_residues_only[0]), f"Length of auth residues and predictions do not match for {protein_id}: {len(auth_residues_only[0])} vs {len(predictions)}"
    predicted_binding_residues = auth_residues_only[0][predictions]
    predicted_smoothed_binding_residues = auth_residues_only[0][smoothed_predictions_array]
    true_positives = binding_residues.intersection([str(i) for i in predicted_binding_residues])
    
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)
    # Show smoothing predictions in green
    # if len(predicted_smoothed_binding_residues) > 0:
    #     cmd.color('green',f'{protein_id} and resi {"+".join([str(i) for i in predicted_smoothed_binding_residues])}')

    # Show regular predictions in blue
    if len(predicted_binding_residues) > 0:
        cmd.color('blue',f'{protein_id} and resi {"+".join([str(i) for i in predicted_binding_residues])}')
        cmd.show('surface',f'{protein_id} and resi {"+".join([str(i) for i in predicted_binding_residues])}')
    # cmd.color('blue', f'{protein_id} and resi {"+".join([str(i) for i in binding_residues])}')
    # if len(true_positives) > 0:
    #     cmd.color('green', f'{protein_id} and resi {"+".join([str(i) for i in true_positives])}')
    # cmd.show('surface', protein_id)
    decision = input(">Press Enter for the next protein...\n")
    if decision.lower() == 'q':
        break
# 3n4u
# 7o1i

 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 You clicked /7e5qB/QA/B/NA`419/NA
 Selector: selection "sele" defined with 1 atoms.
PyMOL>color grey
 Executive: Colored 3100 atoms.
PyMOL>hide surface
PyMOL>show surface, sele
PyMOL>color blue, sele
 Executive: Colored 51 atoms.
PyMOL>color green, 7e5qB within 15 of sele
 Executive: Colored 1219 atoms.
PyMOL>show surface, sele
PyMOL>show surface, 7e5qB within 15 of sele
PyMOL>hide surface
PyMOL>color grey
 Executive: Colored 3100 atoms.
PyMOL>show surface, 7e5qB within 7 of sele
PyMOL>color green, 7e5qB within 7 of sele
 Executive: Colored 376 atoms.
PyMOL>color blue, sele
 Executive: Colored 51 atoms.
PyMOL>color RED, 7e5qB within 7 of sele
 Executive: Colored 376 atoms.
PyMOL>color green, 7e5qB within 7 of sele
 Executive: Colored 376 atoms.
PyMOL>color blue, sele
 Executive: Colored 51 atoms.
